In [ ]:
# Copyright (c) 2022, salesforce.com, inc.
# All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

In this notebook, we run the NLP model and then save the model output, as well as the SHAP values to explain the model's behavior on each instance.

We will need to have access to the pre-trained model in this example. You can either download [the pretrained model](https://storage.googleapis.com/sfr-isea-research/mnli_government.tar.gz) to the runtime, or upload your own model to the Google Drive and then load it.

To read or write files on Google Drive, you can run the code below to mount your Google Drive in the notebook:

```
from google.colab import drive
drive.mount('/content/drive')
```

Then you can specify the path to the file on the Google drive by a path string starting with "/content/drive/MyDrive/"

# 1. Run the pretrained model and save model output

In [ ]:
!pip install transformers==4.10.3

     |████████████████████████████████| 2.8 MB 3.2 MB/s 
     |████████████████████████████████| 596 kB 15.7 MB/s 
     |████████████████████████████████| 895 kB 11.7 MB/s 
     |████████████████████████████████| 67 kB 3.7 MB/s 
     |████████████████████████████████| 3.3 MB 13.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Below you need to specify the path to the pretrained model, a distilbert model trained based on the government genre. The model we described in the paper can be found at https://storage.googleapis.com/sfr-isea-research/mnli_government.tar.gz 

In this notebook, we put the checkpoint folder for the model on the google drive. We mount the google drive as shown in the code block above to access the pretrained model.

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("<specify your path>/checkpoints_government/checkpoint-26106")



In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
'''try to run the model and check the output format and content'''
premise = "A woman with a green headscarf, blue shirt and a very big grin."
hypothesis = "The woman is very happy."

input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
output = model(input_ids)


In [ ]:
import numpy as np

logits = output[0].detach().cpu().numpy()
    
predictions = np.argmax(logits, axis=1).flatten()

In [ ]:
probs = output[0].softmax(dim=1)

label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
for i, lab in label_map.items():
    print('{lab} probability: {prob:0.2f}%'.format(lab=lab, prob=probs[0][i] * 100))


contradiction probability: 0.22%
neutral probability: 98.72%
entailment probability: 1.07%


Below we download the MNLI data set, travel genre, to test the robustness of the pretrained model on a out-of-distribution dataset.

In [ ]:
!pip install datasets

In [ ]:
'''load snli data'''
from datasets import load_dataset

dataset = load_dataset('multi_nli', split='validation_matched')

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/227M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


In [ ]:
genre_to_test = ['travel']

data_by_genre = {
    'travel': dataset.filter(lambda x: x['genre']=='travel'),
}

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
'''check the input format'''
data_by_genre['travel'][0]

{'genre': 'travel',
 'hypothesis': "Most of Mrinal Sen's work can be found in European collections.",
 'hypothesis_binary_parse': "( ( Most ( of ( ( Mrinal ( Sen 's ) ) work ) ) ) ( ( can ( be ( found ( in ( European collections ) ) ) ) ) . ) )",
 'hypothesis_parse': "(ROOT (S (NP (NP (JJS Most)) (PP (IN of) (NP (NP (NNP Mrinal) (NNP Sen) (POS 's)) (NN work)))) (VP (MD can) (VP (VB be) (VP (VBN found) (PP (IN in) (NP (JJ European) (NNS collections)))))) (. .)))",
 'label': 1,
 'pairID': '93958n',
 'premise': "Calcutta seems to be the only other production center having any pretensions to artistic creativity at all, but ironically you're actually more likely to see the works of Satyajit Ray or Mrinal Sen shown in Europe or North America than in India itself.",
 'premise_binary_parse': "( ( ( ( ( Calcutta ( seems ( to ( be ( ( the ( only ( other ( production center ) ) ) ) ( ( having ( any pretensions ) ) ( to ( ( artistic creativity ) ( at all ) ) ) ) ) ) ) ) ) , ) but ) ( ironically ( 

In [ ]:
def pred(premise, hypothesis):
  input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
  output = model(input_ids)

  logits = output[0].detach().cpu().numpy()
  prediction = np.argmax(logits, axis=1).flatten()[0]

  return prediction

In [ ]:
''' 
  To save the model output, first check whether the folder exists. 
  You may want to specify your own path for the output file.
'''

import os
path = "<specify the path>/mnli_government_travel/"
if not os.path.exists(path):
  os.makedirs(path)

In [ ]:
'''
  Run the model and save the model output.
'''
import pandas as pd

for genre in genre_to_test:
  print("test on", genre)
  data = data_by_genre[genre]
  all_gt = []
  all_pred = []
  for item in data:
    all_pred.append(pred(item['premise'], item['hypothesis']))
    all_gt.append(item['label'])
  data_to_output = pd.DataFrame()
  data_to_output['y_pred'] = all_pred
  data_to_output['y_gt'] = all_gt

  data_to_output.to_csv(path+"model_output.csv", index=None)

  # calculate accuracy
  print("accuracy", (data_to_output['y_pred'] == data_to_output['y_gt']).astype(int).sum()/len(all_gt))

test on travel
accuracy 0.7206477732793523


# 2. Generate SHAP Values
Below we describe how we generate SHAP values for each instance, and then save the top 3 tokens which have the most "influence" on the model output for each class for each instance.

In [ ]:
!pip install shap

     |████████████████████████████████| 564 kB 2.6 MB/s 


In [ ]:
import shap

In [ ]:
import torch
import scipy as sp

In [ ]:
def f(x): 
    outputs = []
    for _x in x:
        encoding = torch.tensor([tokenizer.encode(_x)])
        output = model(encoding)[0].detach().cpu().numpy() 
        outputs.append(output[0])
    outputs = np.array(outputs)
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val


explainer = shap.Explainer(f, tokenizer)

In [ ]:
doc_list = []
for item in data_by_genre['travel']:
  encoded = tokenizer(item['premise'], item['hypothesis'])['input_ids'][1:-1] # ignore the start and end tokens, since tokenizer will naturally add them
  decoded = tokenizer.decode(encoded)
  doc_list.append(decoded)

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'text':doc_list,'label':data_by_genre['travel']['label']}) 

In [ ]:
'''
  Generate the SHAP values. May take long time.
'''
shap_values = explainer(df['text'])

In [ ]:
import numpy as np

top_tokens = []
for idx in range(shap_values.values.shape[0]):
  token_idx = []
  # 3 possible prediction values
  for pred_val in range(3):
    temp_list = np.abs(shap_values.values[idx][:, pred_val])
    order = np.argsort(temp_list)
    '''save the top three tokens with the highest absolute SHAP value'''
    largest_indices = order[::-1][:3]
    token_idx.append([
      {"token": shap_values.data[idx][largest_indices[0]], 'val': shap_values.values[idx][largest_indices[0]][pred_val]},
      {"token": shap_values.data[idx][largest_indices[1]], 'val': shap_values.values[idx][largest_indices[1]][pred_val]},
      {"token": shap_values.data[idx][largest_indices[2]], 'val': shap_values.values[idx][largest_indices[2]][pred_val]},
    ])
  top_tokens.append(token_idx)


In [ ]:
import json

with open('<specify your path>/mnli_government_travel/shap_values.json', 'w') as output:
  output.write(json.dumps({"top_tokens": top_tokens}))